# Constraints

The constraints transform an ill-posed problem into a well-posed problem. So, we formulate a constrained non-linear inversion by summing constraints to the misfit function $\Psi (\textbf{m})$. 

\begin{equation}
\Gamma (\textbf{m}) = \Psi (\textbf{m}) + \mu \sum\limits^{6}_{j=1}\alpha_{j}\varphi_{j}(\textbf{m})
\end{equation}

where $\textbf{m}$ is the parameter vector, $\varphi_{j}$ is the $j$th constraint, $\alpha_{j}$ is the $j$-th is a positive scalar that defines the weight of the $j$-th constraint and $\mu$ is a positive scalar called regularization parameter. To solve this problem we will use the Levenberg-Marquadt method.

In this code, we impose six constraints to an Hessian matrix and parameter vector. The functions are tested in some special cases.

### Import required library

In [2]:
import numpy as np
import polyprism_tests as tests

/home/vital/anaconda2/envs/radial-mag/lib/python2.7/site-packages/fatiando/vis/mpl.py:76: UserWarning: This module will be removed in v0.6. We recommend the use of matplotlib.pyplot module directly. Some of the fatiando specific functions will remain.
  "specific functions will remain.")


### Import my functions from an external file

In [3]:
import mag_polyprism_functions as mfun

### Input

In [4]:
M = 4 # number of vertices
L = 3 # number of prisms
P = L*(M+2) # number of parameters

In [5]:
alpha = 1.0 # weight

In [6]:
H1 = np.zeros((P,P)) # Hessian matrix
H2 = np.zeros_like(H1)
H3 = np.zeros_like(H1)
H4 = np.zeros_like(H1)
H5 = np.zeros_like(H1)
H6 = np.zeros_like(H1)

In [7]:
m1 = np.arange(1., P+1., 1.) # gradient of parameter vector
m2 = np.copy(m1)
m3 = np.copy(m1)
m4 = np.copy(m1)
m5 = np.copy(m1)
m6 = np.copy(m1)

In [8]:
print m1

[  1.   2.   3.   4.   5.   6.   7.   8.   9.  10.  11.  12.  13.  14.  15.
  16.  17.  18.]


In [9]:
ml = np.zeros(M+2) # constant vector input for phi3
m2l = np.zeros(2) # constant vector input for phi4

### Constraints

Here we describe the constraints and the result from its functions.

### (1) Smoothness constraint on the adjacent radial distances within each prism $\varphi_{1}(\textbf{m})$

Smoothness constraint on the estimated radial distances within each prism. This constraint imposes that the $j$-th radial distance of the $k$-th prism $r^{k}_{j}$ must be close to the adjacent one $r^{k}_{j+1}$ within the same $k$-th prism. It prevents descontinuities on the estimated radial distances within each prism and imposes a cylindrical shape to the prisms. So, the squared Euclidian Norm of the difference between the adjacent radial distances must be minimun. This constraint can be described by

\begin{equation}
\varphi_{1}(\textbf{m}) = \sum\limits^{L}_{k=1}\left[\left(r^{k}_{M}-r^{k}_{1}\right)^2 + \sum\limits^{M-1}_{j=1}\left(r^{k}_{j}-r^{k}_{j+1}\right)^2\right]
\end{equation}

The geometric representation of the constraint $\varphi_{1}(\textbf{m})$ is presented in the figure below:

<img src="Constraint_phi1.png",width=500>

For simplicity we rewrite the equation above with matrix notation. This constraint will be:

\begin{equation}
\varphi_{1}(\textbf{m}) = \textbf{m}^{T}\textbf{R}^{T}_{1}\textbf{R}_{1}\textbf{m}
\end{equation}

where

\begin{equation}
\textbf{R}_{1} = 
\begin{bmatrix}
\textbf{R}^{\sharp} & 0 & \cdots & 0 \\
0 & \textbf{R}^{\sharp} &  \cdots & 0 \\
\vdots & \vdots & \ddots & \vdots \\
0 & 0 &  \cdots & \textbf{R}^{\sharp} \\
\end{bmatrix}_{ML\times P}
\end{equation}

while

\begin{equation}
\textbf{R}^{\sharp} = 
\begin{bmatrix}
1 & -1 & 0 & \cdots & 0 & 0 & 0 & 0 \\
0 & 1 & -1 & \cdots & 0 & 0 & 0 & 0 \\
\vdots & \vdots & \vdots & & \vdots & \vdots & \vdots & \vdots\\
0 & 0 & 0 & \cdots & 1 & -1 & 0 & 0 \\
-1 & 0 & 0 & \cdots & 0 & 1 & 0 & 0 \\
\end{bmatrix}_{M(L-1)\times P}
\end{equation}

To run the Levenberg-Marquadt algorithm we will need the Laplacian and the gradient of $\varphi_{1}(\textbf{m})$:

\begin{equation}
\nabla\varphi_{1}(\textbf{m}) = \textbf{R}^{T}_{1}\textbf{R}_{1}\textbf{m}
\end{equation}
\begin{equation}
\nabla^2\varphi_{1}(\textbf{m}) = \textbf{R}^{T}_{1}\textbf{R}_{1}
\end{equation}

In [10]:
# m1 = gradient input + gradient of phi1
grad1 = mfun.gradient_phi_1(M,L,m1,alpha)
# H1 = Hessian input + Hessian of phi1
H1 = mfun.Hessian_phi_1(M,L,H1,alpha)

In [11]:
print m1

[  1.   2.   3.   4.   5.   6.   7.   8.   9.  10.  11.  12.  13.  14.  15.
  16.  17.  18.]


In [12]:
print grad1

[ -3.   2.   3.   8.   5.   6.   3.   8.   9.  14.  11.  12.   9.  14.  15.
  20.  17.  18.]


In [13]:
print H1

[[ 2. -1.  0. -1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [-1.  2. -1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0. -1.  2. -1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [-1.  0. -1.  2.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  2. -1.  0. -1.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0. -1.  2. -1.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0. -1.  2. -1.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0. -1.  0. -1.  2.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  2. -1.  0. -1.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.

In [14]:
tests.test_Hessian_phi_1()

In [15]:
phi1 = np.dot(np.dot(H1,m1),m1)

In [16]:
print phi1

36.0


### (2) Smoothness constraint on the adjacent radial distances of adjacent prism $\varphi_{2}(\textbf{m})$

This constraint imposes that the squared Euclidian norm of diference between adjacent radial distances ($r_j^k$ e $r_j^{k+1}$) within vertically adjacent prisms must be minimum. It prevents abrupt descontinuities on the estimated adjacent radial distances within adjacent prism that forces all estimated prism to have a similar shape. Mathematically, it can be described by

\begin{equation}
\varphi_{2}(\textbf{m}) = \sum\limits^{L-1}_{k=1}\left[\sum\limits^{M}_{j=1}\left(r^{k+1}_{j}-r^{k}_{j}\right)^2\right]
\end{equation}

The geometric representation of the constraint $\varphi_{2}(\textbf{m})$ is presented in the figure below:

<img src="Constraint_phi2.png",width=400>

For simplicity we rewrite the equation above with matrix notation. This constraint will be:

\begin{equation}
    \varphi_{2}(\textbf{m}) = \textbf{m}^{T}\textbf{R}^{T}_{2}\textbf{R}_{2}\textbf{m}
\end{equation}

where

\begin{equation}
    \textbf{R}_{2} = 
    \begin{bmatrix}
        \textbf{R}^{-}_{2} & \textbf{R}^{+}_{2} & 0 & \cdots & 0 & 0 \\
        0 & \textbf{R}^{-}_{2} & \textbf{R}^{+}_{2} & \cdots & 0 & 0 \\
        \vdots & \vdots & \vdots & & \vdots & \vdots \\
        0 & 0 & 0 & \cdots & \textbf{R}^{-}_{2} & \textbf{R}^{+}_{2} \\
    \end{bmatrix}_{M(L-1)\times P}
\end{equation}

while

\begin{equation}
    \textbf{R}^{-}_{2} = 
    \begin{bmatrix}
        -1 & 0 & \cdots & 0 & 0 & 0 \\
        0 & -1 & \cdots & 0 & 0 & 0 \\
        \vdots & \vdots & \ddots & \vdots & \vdots & \vdots \\
        0 & 0 & \cdots & -1 & 0  & 0 \\
    \end{bmatrix}_{M\times (M+2)}
\end{equation}

and

\begin{equation}
    \textbf{R}^{+}_{2} = 
     \begin{bmatrix}
        1 & 0 & \cdots & 0 & 0 & 0 \\
        0 & 1 & \cdots & 0 & 0 & 0 \\
        \vdots & \vdots & \ddots & \vdots & \vdots & \vdots \\
        0 & 0 & \cdots & 1 & 0 & 0 \\
    \end{bmatrix}_{M\times (M+2)}
\end{equation}

To run the Levenberg-Marquadt algorithm we will need the Laplacian and the gradient of $\varphi_{2}(\textbf{m})$:

\begin{equation}
\nabla\varphi_{2}(\textbf{m}) = \textbf{R}^{T}_{2}\textbf{R}_{2}\textbf{m}
\end{equation}
\begin{equation}
\nabla^2\varphi_{2}(\textbf{m}) = \textbf{R}^{T}_{2}\textbf{R}_{2}
\end{equation}

In [17]:
# m2 = gradient input + gradient of phi2
grad2 = mfun.gradient_phi_2(M,L,m2,alpha)
# H2 = Hessian input + Hessian of phi2
H2 = mfun.Hessian_phi_2(M,L,H2,alpha) 

In [18]:
print grad2

[ -5.  -4.  -3.  -2.   5.   6.   7.   8.   9.  10.  11.  12.  19.  20.  21.
  22.  17.  18.]


In [19]:
print H2

[[ 1.  0.  0.  0.  0.  0. -1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  1.  0.  0.  0.  0.  0. -1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  1.  0.  0.  0.  0.  0. -1.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  1.  0.  0.  0.  0.  0. -1.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [-1.  0.  0.  0.  0.  0.  2.  0.  0.  0.  0.  0. -1.  0.  0.  0.  0.  0.]
 [ 0. -1.  0.  0.  0.  0.  0.  2.  0.  0.  0.  0.  0. -1.  0.  0.  0.  0.]
 [ 0.  0. -1.  0.  0.  0.  0.  0.  2.  0.  0.  0.  0.  0. -1.  0.  0.  0.]
 [ 0.  0.  0. -1.  0.  0.  0.  0.  0.  2.  0.  0.  0.  0.  0. -1.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0. -1.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.

In [20]:
tests.test_Hessian_phi_2()

In [21]:
phi2 = np.dot(np.dot(H2,m2),m2)

In [22]:
print phi2

288.0


### (3) The constraint of outcropping source $\varphi_{3}(\textbf{m})$

This constraint imposes that the estimated horizontal face of the shallowest prism must be close to the known outcropping boundary. The outcropping boundary must be describe by a polygon, whose radial distances and the Cartesian coordinates of the origin are $r_j^0$, $x_0^0$ and $y_0^0$. The shallowest prism is described by $r_j^1$, $x_j^1$ and $y_j^0$. So, the constraint imposes that a minimum value to the squared Euclidian norm between $r_j^0$ and $r_j^1$ and the squared Euclidian norm between the horizontal coordinates ($x_0^1$,$y_0^1$) and ($x_0^0$,$y_0^0$):

\begin{equation}
\varphi_{3}(\textbf{m}) = \left[\left(x_{0}^{1} - x_{0}^{0}\right)^2 + \left(y_{0}^{1} - y_{0}^{0}\right)^2 + \sum\limits^{M}_{j=1}\left(r^{1}_{j}-r^{0}_{j}\right)^2\right]
\end{equation}

The geometric representation of the constraint $\varphi_{3}(\textbf{m})$ is presented in the figure below:

<img src="Constraint_phi3.png",width=400>

For simplicity we rewrite the equation above with matrix notation. This constraint will be:

\begin{equation}
\varphi_{3}(\textbf{m}) = \left(\textbf{A}\textbf{m} - \textbf{m}"\right)^{T}\left(\textbf{A}\textbf{m} - \textbf{m}"\right)
\end{equation}

where

\begin{equation}
\textbf{m}" = 
\begin{bmatrix}
r_{1}^{0} \\
r_{2}^{0} \\
\vdots \\
r_{M}^{0} \\
x_{0}^{0} \\
y_{0}^{0}
\end{bmatrix}_{P\times 1}
\end{equation}

and

\begin{equation}
\textbf{A} = 
\begin{bmatrix}
\textbf{I} & 0 \\
\end{bmatrix}_{(M+2)\times P}
\end{equation}

while

$\textbf{I}$ is the identity of order $M+2$.

To run the Levenberg-Marquadt algorithm we will need the Laplacian and the gradient of $\varphi_{3}(\textbf{m})$:

\begin{equation}
\nabla\varphi_{3}(\textbf{m}) = \textbf{A}^{T}\left(\textbf{A}\textbf{m} - \textbf{m}"\right)
\end{equation}
\begin{equation}
\nabla^2\varphi_{3}(\textbf{m}) = \textbf{A}^{T}\textbf{A}
\end{equation}

In [23]:
# grad = gradient input + gradient of phi3
grad3 = mfun.gradient_phi_3(M,L,m3,ml,alpha)
# H3 = Hessian input + Hessian of phi3
H3 = mfun.Hessian_phi_3(M,L,H3,alpha) 

In [24]:
print grad3

[  2.   4.   6.   8.  10.  12.   7.   8.   9.  10.  11.  12.  13.  14.  15.
  16.  17.  18.]


In [25]:
print H3

[[ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.

In [26]:
tests.test_Hessian_phi_3()

In [27]:
phi3 = np.dot(np.dot(H3,m3),m3)

In [28]:
print phi3

364.0


### (4) The constraint of horizontal location of outcropping source $\varphi_{4}(\textbf{m})$

This constraint is an special case from $\varphi_{3}(\textbf{m})$ that only the Cartesian coordinates of the origin of th outcropping body are known. So, this constraint imposes that the squared Euclidian norm between the ($x_0^1$,$y_0^1$) and ($x_0^0$,$y_0^0$):

\begin{equation}
\varphi_{4}(\textbf{m}) = \left[\left(x_{0}^{1} - x_{0}^{0}\right)^2 + \left(y_{0}^{1} - y_0^0\right)^2\right]
\end{equation}

The geometric representation of the constraint $\varphi_{4}(\textbf{m})$ is presented in the figure below:

<img src="Constraint_phi4.png",width=400>

For simplicity we rewrite the equation above with matrix notation. This constraint will be:

\begin{equation}
\varphi_{4}(\textbf{m}) = \left(\textbf{B}\textbf{m} - \textbf{m}'\right)^{T}\left(\textbf{B}\textbf{m} - \textbf{m}'\right)
\end{equation}

where
\begin{equation}
\textbf{B} = 
\begin{bmatrix}
\textbf{B}^{\sharp} & 0 \\
\end{bmatrix}_{2\times P}
\end{equation}

while

\begin{equation}
\textbf{B}^{\sharp} = 
\begin{bmatrix}
0 & \cdots & 0 & 1 & 0 \\
0 & \cdots & 0 & 0 & 1
\end{bmatrix}_{2\times (M+2)}
\end{equation}

and

\begin{equation}
\textbf{m}' = 
\begin{bmatrix}
x^0_0 \\
y^0_0
\end{bmatrix}_{2\times 1}
\end{equation}

To run the Levenberg-Marquadt algorithm we will need the Laplacian and the gradient of $\varphi_{4}(\textbf{m})$:

\begin{equation}
\nabla\varphi_{4}(\textbf{m}) = \textbf{B}^{T}\left(\textbf{B}\textbf{m} - \textbf{m}'\right)
\end{equation}
\begin{equation}
\nabla^2\varphi_{4}(\textbf{m}) = \textbf{B}^{T}\textbf{B}
\end{equation}

In [29]:
# grad4 = gradient input + gradient of phi4
grad4 = mfun.gradient_phi_4(M,L,m4,m2l,alpha) 
# H4 = Hessian input + Hessian of phi4
H4 = mfun.Hessian_phi_4(M,L,H4,alpha) 

In [30]:
print grad4

[  1.   2.   3.   4.  10.  12.   7.   8.   9.  10.  11.  12.  13.  14.  15.
  16.  17.  18.]


In [31]:
print H4

[[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.

In [32]:
tests.test_Hessian_phi_4()

In [33]:
phi4 = np.dot(np.dot(H4,m4),m4)

In [34]:
print phi4

244.0


### (5) Smoothness constraint on the origins of adjacent prisms $\varphi_{5}(\textbf{m})$

This constraint imposses a minimum value to the difference between horizontal Cartesian coordinates of vertically adjacent prisms. It prevents descontinuities between estimated horizontal Cartesian coordinates of vertically adjacent prisms. This constraint forces smooth horizontal displacements between all vertically adjacent prisms. It cand be described by

\begin{equation}
\varphi_{5}(\textbf{m}) = \sum\limits^{L-1}_{k=1}\left[\left(x_{0}^{k+1} - x_{0}^{k}\right)^2 + \left(y_{0}^{k+1} - y_{0}^{k}\right)^2 \right]
\end{equation}

The geometric representation of the constraint $\varphi_{5}(\textbf{m})$ is presented in the figure below:

<img src="Constraint_phi5.png",width=400>

For simplicity we rewrite the equation above with matrix notation. This constraint will be:

\begin{equation}
\varphi_{5}(\textbf{m}) = \textbf{m}^{T}\textbf{R}^{T}_{5}\textbf{R}_{5}\textbf{m}
\end{equation}

where
\begin{equation}
\textbf{R}_{5} = 
\begin{bmatrix}
\textbf{R}^{-}_{5} & \textbf{R}^{+}_{5} & 0 & \cdots & 0 & 0 \\
0 & \textbf{R}^{-}_{5} & \textbf{R}^{+}_{5} & \cdots & 0 & 0 \\
\vdots & \vdots & \vdots & & \vdots & \vdots \\
0 & 0 & 0 & \cdots & \textbf{R}^{-}_{5} & \textbf{R}^{+}_{5} \\
\end{bmatrix}_{2(L-1)\times P}
\end{equation}

while

\begin{equation}
\textbf{R}^{-}_{5} = 
\begin{bmatrix}
0 & 0 & \cdots & 0 & -1 & 0 \\
0 & 0 & \cdots & 0 & 0 & -1 \\
\end{bmatrix}_{2\times (M+2)}
\end{equation}

and

\begin{equation}
\textbf{R}^{+}_{5} = 
\begin{bmatrix}
0 & 0 & \cdots & 0 & 1 & 0 \\
0 & 0 & \cdots & 0 & 0 & 1 \\
\end{bmatrix}_{2\times (M+2)}
\end{equation}

To run the Levenberg-Marquadt algorithm we will need the Laplacian and the gradient of $\varphi_{5}(\textbf{m})$:

\begin{equation}
\nabla\varphi_{5}(\textbf{m}) = \textbf{R}^{T}_{5}\textbf{R}_{5}\textbf{m}
\end{equation}
\begin{equation}
\nabla^2\varphi_{5}(\textbf{m}) = \textbf{R}^{T}_{5}\textbf{R}_{5}
\end{equation}

In [35]:
# grad5 = gradient input + gradient of phi5
grad5 = mfun.gradient_phi_5(M,L,m5,alpha)
# H5 = Hessian input + Hessian of phi5
H5 = mfun.Hessian_phi_5(M,L,H5,alpha) 

In [36]:
print grad5

[  1.   2.   3.   4.  -1.   0.   7.   8.   9.  10.  11.  12.  13.  14.  15.
  16.  23.  24.]


In [37]:
print H5

[[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  1.  0.  0.  0.  0.  0. -1.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0. -1.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0. -1.  0.  0.  0.  0.  0.  2.  0.  0.  0.  0.  0. -1.  0.]
 [ 0.  0.  0.  0.  0. -1.  0.  0.  0.  0.  0.  2.  0.  0.  0.  0.  0. -1.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.

In [38]:
tests.test_Hessian_phi_5()

In [39]:
phi5 = np.dot(np.dot(H5,m5),m5)

In [40]:
print phi5

144.0


### (6) Minimum Euclidian norm constraint on the adjacent radial distances within each prism $\varphi_{6}(\textbf{m})$

This constraint imposes that the squared Euclidian norm of the radial distances must be minimum. It implies that the estimated radial distances to be close to zero. Mathematically, it can be described as:

\begin{equation}
\varphi_{6}(\textbf{m}) = \sum\limits^{L}_{k=1}\sum\limits^{M}_{j=1}\left(r_{j}^{k}\right)^2
\end{equation}

The geometric representation of the constraint $\varphi_{6}(\textbf{m})$ is presented in the figure below:

<img src="Constraint_phi6.png",width=400>

For simplicity we rewrite the equation above with matrix notation. This constraint will be:

\begin{equation}
\varphi_{6}(\textbf{m}) = \textbf{m}^{T}\textbf{C}^{T}\textbf{C}\textbf{m}
\end{equation}

where

\begin{equation}
\textbf{C} = 
\begin{bmatrix}
\textbf{C}^{\sharp} & 0 & \cdots & 0 \\
0 & \textbf{C}^{\sharp} &  \cdots & 0 \\
\vdots & \vdots & \ddots & \vdots \\
0 & 0 &  \cdots & \textbf{C}^{\sharp} \\
\end{bmatrix}_{P\times P}
\end{equation}

while



\begin{equation}
\textbf{C}^{\sharp} = 
\begin{bmatrix}
1 & & & & \\
 & \ddots &  & &  \\
 &  & 1 & & \\
 &  & & 0 &  \\
 &  & & & 0 \\
\end{bmatrix}_{(M+2)\times (M+2)}
\end{equation}

To run the Levenberg-Marquadt algorithm we will need the Laplacian and the gradient of $\varphi_{6}(\textbf{m})$:

\begin{equation}
\nabla\varphi_{5}(\textbf{m}) = \textbf{C}^{T}\textbf{C}\textbf{m}
\end{equation}
\begin{equation}
\nabla^2\varphi_{5}(\textbf{m}) = \textbf{C}^{T}\textbf{C}
\end{equation}

In [41]:
# grad6 = gradient input + gradient of phi6
grad6 = mfun.gradient_phi_6(M,L,m6,alpha)
# H6 = Hessian input + Hessian of phi6
H6 = mfun.Hessian_phi_6(M,L,H6,alpha) 

In [42]:
print grad6

[  2.   4.   6.   8.   5.   6.  14.  16.  18.  20.  11.  12.  26.  28.  30.
  32.  17.  18.]


In [43]:
print H6

[[ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.

In [44]:
tests.test_Hessian_phi_6()

In [45]:
phi6 = np.dot(np.dot(H6,m6),m6)

In [46]:
print phi6

1170.0


In [47]:
fi6 = mfun.diags_phi_6(M,L,alpha)

In [48]:
print fi6

[ 1.  1.  1.  1.  0.  0.  1.  1.  1.  1.  0.  0.  1.  1.  1.  1.  0.  0.]
